In [1]:
from sympy.physics.mechanics import (
    dynamicsymbols, ReferenceFrame, Point, RigidBody, Particle, System, Force
)
from sympy import zeros, symbols
import sympy
import numpy as np
import sympy as sp
import sympy.physics.mechanics as me
from IPython.display import display
from suspycious import Model
import suspycious.components as scmp
from pathway import LinearPathway
from utils import add_points, get_tension_dirs, give_deltas, give_tensions, get_linpaths, get_wire_dir, apply_force
from utils import (
    add_points, make_wire_bodies, get_points, get_deltas, get_linpaths,
    get_wires, get_forces, deltas_dict, linpaths_dict, add_wire_info,
    get_force_wire, apply_force_on_body, apply_gravity
)

In [2]:
d1, d2, n1, n2, w1, w2 = symbols('d1 d2 n1 n2 w1 w2', real=True, positive=True)
d3, n3, w3 = symbols('d3 n3 w3', real=True, positive=True)
d4, n4, w4 = symbols('d4 n4 w4', real=True, positive=True)
d5, n5, w5 = symbols('d5 n5 w5', real=True, positive=True)
d6, n6, w6 = symbols('d6 n6 w6', real=True, positive=True)
d7, n7, w7 = symbols('d7 n7 w7', real=True, positive=True)
d8, d9 = symbols('d8 d9', real=True, positive=True)

l1, l2, l3, l4, l2a = symbols('l1 l2 l3 l4 l_2a', positive=True)

In [3]:
def set_up_bodies():
    model2 = Model()
    S = model2.add(scmp.RigidBody("S"))
    B = model2.add(scmp.RigidBody("B"))
    C = model2.add(scmp.RigidBody("C"))
    D = model2.add(scmp.RigidBody("D"))
    F = model2.add(scmp.RigidBody("F"))
    
    S.set_global_position(0,0,0)
    B.set_global_position(0,0, -l1-d1)
    C.set_global_position(0,0, -l1 -d1-d2-l2-d3)
    D.set_global_position(0,0, -l1 -d1-d2-l2-d3-d4-l3-d5)
    F.set_global_position(0,0, -l1 -d1-d2-l2-d3-d4-l3-d5-d6-l4-d7)
    S.set_global_orientation(0,0,0)
    B.set_global_orientation(0,0,0)
    C.set_global_orientation(0,0,0)
    D.set_global_orientation(0,0,0)
    F.set_global_orientation(0,0,0)
    
    add_points(body=S, point='P', attachment_points=[w1, n1, 0])
    
    add_points(body=B, point='B', attachment_points=[w1, n1, d1])
    add_points(body=B, point='C', attachment_points=[w2, n2, -d2])
    
    add_points(body=C, point='D', attachment_points=[w2, n2, d3])
    add_points(body=C, point='E', attachment_points=[w3, n3, -d4])
    
    add_points(body=D, point='F', attachment_points=[w3, n3, d5])
    add_points(body=D, point='G', attachment_points=[w4, n4, -d6])
    
    add_points(body=F, point='H', attachment_points=[w4, n4, d7])
    add_points(body=F, point='J', attachment_points=[w5, n5, -d8])

    return S, B, C, D, F, model2

In [4]:
S, B, C, D, F, model2 = set_up_bodies()

/Users/siddharth/Desktop/suspycious/src/suspycious/components/body.py:81: SymPyDeprecationWarning: 

Support for the Body class has been removed, as its functionality is
fully captured by RigidBody and Particle.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-mechanics-body-class
for details.

This has been deprecated since SymPy version 1.13. It
will be removed in a future version of SymPy.

  self.__body = mech.Body(


In [5]:
sys = model2.system

In [11]:
n_violin_modes = 1

In [12]:
bodies_ = [[S, B], [B, C], [C, D], [D, F]]
attach_points_ = [[S.P1, B.B1], [B.C1, C.D1], [C.E1, D.F1], [D.G1, F.H1]]

points_bodies = get_points(
        n=n_violin_modes, num_blocks=4, model=model2,
        bodies=bodies_, attach_points=attach_points_, suspension_body=S)

model2.add_bodies_system()


deltas = deltas_dict(
    points_body=points_bodies, num_blocks=4, model=model2,
    suspension_body=S
)

linpaths = linpaths_dict(
    points_body=points_bodies, num_blocks=4, model=model2
)

num_blocks = 4
all_masses = [
    i.body.mass for j in range(num_blocks)
    for i in points_bodies[f'body{j + 1}']['bodies']
]

all_masses_ = list(dict.fromkeys(all_masses))
wires = get_wires(
    num_blocks=4, point_body=points_bodies,
    deltas=deltas, linpaths=linpaths
)

ks = symbols('k1:5')

n_wire_per_wire = n_violin_modes + 1
body1_force = get_forces(
        n_body=1, n_wire_body=n_wire_per_wire,
        wire_dict=wires, spring_constants=ks, masses=all_masses_
    )
body2_force = get_forces(
    n_body=2, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)
body3_force = get_forces(
    n_body=3, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)
body4_force = get_forces(
    n_body=4, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)

for j in [body1_force, body2_force, body3_force, body4_force]:
    for i in j.keys():
        force = j[i]['force']
        path = j[i]['path']
        points = j[i]['points']
        bodies = j[i]['bodies']
    
        sys.add_loads(path.to_loads(-force, frame=S.global_frame)[0])
        sys.add_loads(path.to_loads(-force, frame=S.global_frame)[1])

def apply_gravity(body):
    g = symbols('g')
    body.Fz = -g*body.body.mass
         
    return

apply_gravity(model2.components['S'])
apply_gravity(model2.components['B'])
apply_gravity(model2.components['C'])
apply_gravity(model2.components['D'])
apply_gravity(model2.components['F'])

Calculating deltas for points between bodies S and B
(S.P1, SB11.com)
(SB11.com, B.B1)
(S.P2, SB12.com)
(SB12.com, B.B2)
(S.P3, SB13.com)
(SB13.com, B.B3)
(S.P4, SB14.com)
(SB14.com, B.B4)
Calculating deltas for points between bodies B and C
(B.C1, BC11.com)
(BC11.com, C.D1)
(B.C2, BC12.com)
(BC12.com, C.D2)
(B.C3, BC13.com)
(BC13.com, C.D3)
(B.C4, BC14.com)
(BC14.com, C.D4)
Calculating deltas for points between bodies C and D
(C.E1, CD11.com)
(CD11.com, D.F1)
(C.E2, CD12.com)
(CD12.com, D.F2)
(C.E3, CD13.com)
(CD13.com, D.F3)
(C.E4, CD14.com)
(CD14.com, D.F4)
Calculating deltas for points between bodies D and F
(D.G1, DF11.com)
(DF11.com, F.H1)
(D.G2, DF12.com)
(DF12.com, F.H2)
(D.G3, DF13.com)
(DF13.com, F.H3)
(D.G4, DF14.com)
(DF14.com, F.H4)
Finding linpath between S.P1 and SB11.com
Finding linpath between SB11.com and B.B1
Finding linpath between S.P2 and SB12.com
Finding linpath between SB12.com and B.B2
Finding linpath between S.P3 and SB13.com
Finding linpath between SB13.com a

In [8]:
kane_0 = model2.extract_statespacesystem()

In [9]:
Amat0, Bmat0 = kane_0.A, kane_0.B

In [10]:
S, B, C, D, F, model2 = set_up_bodies()
sys = model2.system

In [13]:
kane_1 = model2.extract_statespacesystem()

In [14]:
Amat1, Bmat1 = kane_1.A, kane_1.B

In [15]:
Amat1.shape

(252, 252)

In [16]:
Amat1[136,10]

-4*d1*(-2*d1*g*(B.M/4 + BC11.M/4 + BC12.M/4 + BC13.M/4 + BC14.M/4 + C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4)/l1 - g*(B.M/4 + BC11.M/4 + BC12.M/4 + BC13.M/4 + BC14.M/4 + C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4)) + d2*(BC11.M*g + 2*d2*(BC11.M*g + g*(C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4))/l2 + g*(C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4)) + d2*(BC12.M*g + 2*d2*(BC12.M*g + g*(C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4))/l2 + g*(C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF12.M/4 + DF13.M/4 + DF14.M/4 + F.M/4)) + d2*(BC13.M*g + 2*d2*(BC13.M*g + g*(C.M/4 + CD11.M/4 + CD12.M/4 + CD13.M/4 + CD14.M/4 + D.M/4 + DF11.M/4 + DF1

In [20]:
kane_0.E

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,    0,    0,    0,         0,         0,         0,   